This notebook is to try to implement the Quadratic Energy algorithm that is mentioned in this paper: https://www.ifaamas.org/Proceedings/aamas2019/pdfs/p1722.pdf. For the other solving algorithms of the acceptibility semantics, since they follows a similar pipeline, it's easy to development them by starting from what we will have below. 

To validate the algorithm correctness, we use the stock example mentioned in this notebook: https://github.com/nicopotyka/Uncertainpy/blob/master/examples/gradual/gradual_argumentation_examples.ipynb.

In [61]:
import numpy as np

# sample graph: a -> b <- c
vs = np.array([[0, 0, 0], [-1, 0, -1], [0, 0, 0]])
weights = np.array([0.5, 0.8, 0.2])

In [50]:
def aggreg_sum(v_parent: np.ndarray, v_strengh: np.ndarray):
    if v_parent.size != v_strengh.size:
        raise ValueError(f"Size of input vectors should be the same, but now {v_parent.size} and {v_strengh.size}")
    return v_parent @ v_strengh

def infl_pmax(aggreg_strengh: float, weight: float, p: int=2, k: float=1):
    h = lambda x: max(0, x) ** p / (1 + max(0, x) ** p)
    return weight * (1 - h(-aggreg_strengh / k) + h(aggreg_strengh / k))
    
def update(vs_parent: np.ndarray, v_strengh: np.ndarray, v_weight: np.ndarray):
    result = []
    for i in range(vs_parent.shape[0]):
        aggreg_s = aggreg_sum(v_parent=vs_parent[i], v_strengh=v_strengh) 
        updated_strength = infl_pmax(aggreg_strengh=aggreg_s, weight=v_weight[i]) 
        result.append(updated_strength)
    return np.array(result)

In [62]:
update(vs_parent=vs, v_strengh=weights, v_weight=weights)

array([0.5       , 0.53691275, 0.2       ])